In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statistics

from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
import warnings
warnings.filterwarnings("ignore")


In [2]:
# Make sure to use the correct path where your Excel file is located in your Google Drive
Train = pd.read_excel(r"C:\Users\USER\Downloads\train_processed.xlsx")
Test = pd.read_excel(r"C:\Users\USER\Downloads\test_processed.xlsx")

In [4]:
path = "/content/test_processed.xlsx"
Test = pd.read_excel(path)

In [9]:
Train

,Delivery_person_Age,Delivery_person_Ratings,Type_of_order,Type_of_vehicle,Time_taken(min),Distance
0,30,4.7,Drinks,motorcycle,30,17.298
1,35,5.0,Snack,motorcycle,52,14.153
2,26,4.7,Snack,scooter,20,20.181
3,27,5.0,Drinks,scooter,36,10.587
4,36,3.5,Drinks,scooter,33,12.100
...,...,...,...,...,...,...
36125,33,4.7,Buffet,scooter,22,1.490
36126,23,4.5,Snack,scooter,28,13.404
36127,37,4.7,Buffet,scooter,17,9.121
36128,24,4.7,Drinks,motorcycle,19,1.553


In [10]:
Test

,Delivery_person_Age,Delivery_person_Ratings,Type_of_order,Type_of_vehicle,Time_taken(min),Distance
0,29,4.5,Buffet,electric_scooter,20,13.764
1,35,4.3,Drinks,scooter,38,13.682
2,28,4.6,Snack,scooter,22,16.826
3,23,4.9,Drinks,electric_scooter,11,9.175
4,27,4.9,Drinks,scooter,25,9.315
...,...,...,...,...,...,...
9027,29,4.6,Meal,scooter,12,3.041
9028,26,4.9,Drinks,scooter,32,11.008
9029,21,4.8,Drinks,motorcycle,22,4.589
9030,30,4.9,Snack,motorcycle,26,3.116


In [3]:
from sklearn.preprocessing import OneHotEncoder


In [4]:
import pandas as pd

def one_hot_encode(df):
    # Identify categorical columns of type 'object'
    categorical_columns = df.select_dtypes(include='object').columns

    # Apply pd.get_dummies() and merge it with the numeric data in the original df
    df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=False)

    return df_encoded

# Usage example with a DataFrame 'df_train'
Train_encoded = one_hot_encode(Train)
Train_encoded.head()

Test_encoded = one_hot_encode(Test)
Test_encoded.head()


,Delivery_person_Age,Delivery_person_Ratings,Vehicle_condition,multiple_deliveries,Time_taken.min.,Distance,Weatherconditions_Cloudy,Weatherconditions_Fog,Weatherconditions_Sandstorms,Weatherconditions_Stormy,...,Type_of_vehicle_electric_scooter,Type_of_vehicle_motorcycle,Type_of_vehicle_scooter,Festival_NaN,Festival_No,Festival_Yes,City_Metropolitian,City_NaN,City_Semi-Urban,City_Urban
0,33,4.7,1,1.0,40,16.600,False,True,False,False,...,False,False,True,False,True,False,True,False,False,False
1,23,4.7,1,1.0,21,13.788,False,False,True,False,...,False,False,True,False,True,False,True,False,False,False
2,34,4.9,0,NaN,41,10.757,True,False,False,False,...,False,True,False,False,True,False,True,False,False,False
3,35,4.3,1,0.0,38,13.682,True,False,False,False,...,False,False,True,False,True,False,True,False,False,False
4,22,4.8,1,1.0,28,10.867,False,True,False,False,...,False,True,False,False,True,False,True,False,False,False


In [5]:
#test=test.replace(" ","")
Test=Test_encoded.replace('NaN', float(np.nan), regex=True)

#train=train.replace(" ","")
Train=Train_encoded.replace('NaN', float(np.nan), regex=True)

In [6]:
def handle_null_values(df):
    df['Delivery_person_Age'].fillna(np.random.choice(df['Delivery_person_Age']), inplace=True)
    df['multiple_deliveries'].fillna(df['multiple_deliveries'].mode()[0], inplace=True)
    df['Delivery_person_Ratings'].fillna(df['Delivery_person_Ratings'].median(), inplace=True)

handle_null_values(Train)
handle_null_values(Test)

Train.isnull().sum()

Delivery_person_Age                  0
Delivery_person_Ratings              0
Vehicle_condition                    0
multiple_deliveries                  0
Time_taken.min.                      0
Distance                             0
Weatherconditions_Cloudy             0
Weatherconditions_Fog                0
Weatherconditions_Sandstorms         0
Weatherconditions_Stormy             0
Weatherconditions_Sunny              0
Weatherconditions_Windy              0
Road_traffic_density_High            0
Road_traffic_density_Jam             0
Road_traffic_density_Low             0
Road_traffic_density_Medium          0
Road_traffic_density_NaN             0
Type_of_order_Buffet                 0
Type_of_order_Drinks                 0
Type_of_order_Meal                   0
Type_of_order_Snack                  0
Type_of_vehicle_bicycle              0
Type_of_vehicle_electric_scooter     0
Type_of_vehicle_motorcycle           0
Type_of_vehicle_scooter              0
Festival_NaN             

In [7]:
Test_encoded

,Delivery_person_Age,Delivery_person_Ratings,Vehicle_condition,multiple_deliveries,Time_taken.min.,Distance,Weatherconditions_Cloudy,Weatherconditions_Fog,Weatherconditions_Sandstorms,Weatherconditions_Stormy,...,Type_of_vehicle_electric_scooter,Type_of_vehicle_motorcycle,Type_of_vehicle_scooter,Festival_NaN,Festival_No,Festival_Yes,City_Metropolitian,City_NaN,City_Semi-Urban,City_Urban
0,33,4.7,1,1.0,40,16.600,False,True,False,False,...,False,False,True,False,True,False,True,False,False,False
1,23,4.7,1,1.0,21,13.788,False,False,True,False,...,False,False,True,False,True,False,True,False,False,False
2,34,4.9,0,NaN,41,10.757,True,False,False,False,...,False,True,False,False,True,False,True,False,False,False
3,35,4.3,1,0.0,38,13.682,True,False,False,False,...,False,False,True,False,True,False,True,False,False,False
4,22,4.8,1,1.0,28,10.867,False,True,False,False,...,False,True,False,False,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8695,24,4.8,0,0.0,18,1.555,False,True,False,False,...,False,True,False,False,True,False,True,False,False,False
8696,34,4.5,2,1.0,27,19.089,False,False,False,True,...,True,False,False,False,True,False,False,False,False,True
8697,22,4.9,2,0.0,12,3.053,True,False,False,False,...,True,False,False,False,True,False,False,False,False,True
8698,34,4.9,1,1.0,28,9.221,False,False,False,True,...,False,True,False,False,True,False,True,False,False,False


In [8]:
X_train = Train.drop(columns = "Time_taken.min.")
y_train = Train["Time_taken.min."]

In [9]:
X_test = Test.drop(columns = "Time_taken.min.")
y_test = Test["Time_taken.min."]

In [10]:
from sklearn.linear_model import LinearRegression


In [44]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [45]:
# Making predictions
y_pred = lr.predict(X_test)
from sklearn.metrics import r2_score

# Calculate R^2 the coefficient of determination
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")


R-squared: 0.6029848538063738


Lasso Regression

In [47]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


In [48]:
param_grid = {
    'alpha': np.logspace(-4, -0.5, 30)  # Adjust range based on your specific needs
}

lasso = Lasso(max_iter=10000, tol=0.01)  # Increase max_iter and adjust tol if convergence issues occur
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=5, estimator=Lasso(max_iter=10000, tol=0.01),
             param_grid={'alpha': array([1.00000000e-04, 1.32035178e-04, 1.74332882e-04, 2.30180731e-04,
       3.03919538e-04, 4.01280703e-04, 5.29831691e-04, 6.99564216e-04,
       9.23670857e-04, 1.21957046e-03, 1.61026203e-03, 2.12611233e-03,
       2.80721620e-03, 3.70651291e-03, 4.89390092e-03, 6.46167079e-03,
       8.53167852e-03, 1.12648169e-02, 1.48735211e-02, 1.96382800e-02,
       2.59294380e-02, 3.42359796e-02, 4.52035366e-02, 5.96845700e-02,
       7.88046282e-02, 1.04049831e-01, 1.37382380e-01, 1.81393069e-01,
       2.39502662e-01, 3.16227766e-01])},
             scoring='neg_mean_squared_error', verbose=1)

In [49]:
best_lasso = grid_search.best_estimator_
y_pred = best_lasso.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Best alpha parameter: ", grid_search.best_params_)
print("Test Mean Squared Error: ", mse)


Best alpha parameter:  {'alpha': 0.0006995642156712634}
Test Mean Squared Error:  35.2270531826644


In [50]:
# Predict on the training and test data
y_train_pred = best_lasso.predict(X_train)
y_test_pred = best_lasso.predict(X_test)

# Calculate R² score for training and test sets
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print("R² on training data: {:.2f}".format(r2_train))
print("R² on test data: {:.2f}".format(r2_test))


R² on training data: 0.60
R² on test data: 0.60


XGB 1 - no tuning

In [51]:
import xgboost as xg

xgb_r = xg.XGBRegressor(objective ='reg:linear', n_estimators=100, seed = 123)
xgb_r.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, objective='reg:linear', ...)

In [53]:
def model_eval(model):
    # Lets calculate some metrics such as R2 score, RSS and RMSE
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    metric = []
    r2_train_lr = r2_score(y_train, y_pred_train)
    print("R2 score for train:", r2_train_lr)
    metric.append(r2_train_lr)

    r2_test_lr = r2_score(y_test, y_pred_test)
    print("R2 score for test:", r2_test_lr)
    metric.append(r2_test_lr)
    print("--"*20)

    rss1_lr = np.sum(np.square(y_train - y_pred_train))
    print("RSS for train:", rss1_lr)
    metric.append(rss1_lr)

    rss2_lr = np.sum(np.square(y_test - y_pred_test))
    print("RSS for test:", rss2_lr)
    metric.append(rss2_lr)
    print("--"*20)

    mse_train_lr = mean_squared_error(y_train, y_pred_train)
    print("MSE for train:", mse_train_lr)
    metric.append(mse_train_lr)

    mse_test_lr = mean_squared_error(y_test, y_pred_test)
    print("MSE for test:", mse_test_lr)
    metric.append(mse_test_lr)
    print("--"*20)

    rmse_train_lr = mse_train_lr**0.5
    print("RMSE for train:", rmse_train_lr)
    metric.append(rmse_train_lr)

    rmse_test_lr = mse_test_lr**0.5
    print("RMSE for test:", rmse_test_lr)
    metric.append(rmse_test_lr)
    print("--"*20)

    return metric, y_pred_train, y_pred_test

In [54]:
metric, y_pred_train, y_pred_test = model_eval(xgb_r)

R2 score for train: 0.8734255764817572
R2 score for test: 0.8389080286368669
----------------------------------------
RSS for train: 386067.05283227854
RSS for test: 124293.80270666309
----------------------------------------
MSE for train: 11.093243285796177
MSE for test: 14.286643989271619
----------------------------------------
RMSE for train: 3.330652081169118
RMSE for test: 3.779767716311628
----------------------------------------


XGB tuning

In [17]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder

In [18]:
param_grid = {
    "max_depth": [7, 8, 9],
    "min_child_weight": [5, 9],
    "gamma": [0.5, 1, 1.5],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "learning_rate": [0.01, 0.05, 0.1],
    "n_estimators": [100, 200, 300]
}


In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Define the model to use
xgboost_model = xgb.XGBRegressor()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=xgboost_model, param_grid=param_grid,
                           scoring='neg_mean_squared_error', cv=3, verbose=1)

# Perform grid search
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 1458 candidates, totalling 4374 fits


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'gamma': [0.5, 1, 1.5],
                         'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [7, 8, 9], 'min_child_weight': [5, 9],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.6, 0.8, 1.0]},
             scoring='neg_mean_squared_error', verbose=1)

In [20]:
from sklearn.metrics import mean_squared_error, r2_score
# Best parameters found
print("Best parameters:", grid_search.best_params_)

# Best model from grid search
best_model = grid_search.best_estimator_

# Predict using the test dataset
predictions = best_model.predict(X_test)

# Calculate MSE and R^2
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

# Print the metrics
print("Mean Squared Error on Test Set:", mse)
print("R^2 Score on Test Set:", r2)


Best parameters: {'colsample_bytree': 1.0, 'gamma': 0.5, 'learning_rate': 0.05, 'max_depth': 8, 'min_child_weight': 9, 'n_estimators': 200, 'subsample': 1.0}
Mean Squared Error on Test Set: 13.546746317461155
R^2 Score on Test Set: 0.847250895908456


RANDOM FOREST


In [13]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [14]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4, 6, 8, 10],
    'bootstrap': [True]
}


In [15]:
# Initialize the RandomForestRegressor
rf = RandomForestRegressor(random_state=42)

# Setup GridSearchCV or RandomizedSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1, scoring='neg_mean_squared_error')

# Fit the model
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 648 candidates, totalling 3240 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [None, 10, 20, 30],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4, 6, 8, 10],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error', verbose=2)

In [16]:
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Best Parameters:", grid_search.best_params_)
print("Test MSE:", mse)
print("Test R2 Score:", r2)


Best Parameters: {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
Test MSE: 14.2078050849106
Test R2 Score: 0.8397969928004005


KNN

In [67]:
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [68]:
# Initialize the KNN regressor
knn = KNeighborsRegressor()

# Define the hyperparameter grid
param_grid = {
    'n_neighbors': np.arange(1, 30),  # Considering different numbers of neighbors
    'weights': ['uniform', 'distance'],  # Weight types
    'metric': ['euclidean', 'manhattan', 'minkowski']  # Distance metrics
}

In [69]:
# Setup the GridSearchCV
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid,
                           cv=5, verbose=1, scoring='neg_mean_squared_error')

# Execute the grid search
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 174 candidates, totalling 870 fits


GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'metric': ['euclidean', 'manhattan', 'minkowski'],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
                         'weights': ['uniform', 'distance']},
             scoring='neg_mean_squared_error', verbose=1)

In [70]:
# Best parameters found
print("Best parameters:", grid_search.best_params_)

# Best model
best_knn = grid_search.best_estimator_

# Predict on the test data
predictions = best_knn.predict(X_test)

# Calculate MSE and R^2
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

# Print the evaluation metrics
print("Mean Squared Error on Test Set:", mse)
print("R^2 Score on Test Set:", r2)

Best parameters: {'metric': 'manhattan', 'n_neighbors': 29, 'weights': 'uniform'}
Mean Squared Error on Test Set: 61.959177778784166
R^2 Score on Test Set: 0.27959727693358016


In [11]:
import optuna

In [21]:
#### optuna for random forest

In [12]:
def objective(trial):
    # Define the hyperparameters to optimize
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 10, 50, step=5)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 4)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])  # Updated here

    # Model
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )

    # Score model with cross-validation
    score = cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_squared_error")
    mse = -1 * np.mean(score)

    return mse


In [13]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)


[I 2024-04-14 07:17:38,005] A new study created in memory with name: no-name-5849aedf-d32c-4864-92d5-40735d7c4504
[I 2024-04-14 07:18:06,480] Trial 0 finished with value: 14.611247613427016 and parameters: {'n_estimators': 84, 'max_depth': 25, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': None}. Best is trial 0 with value: 14.611247613427016.
[I 2024-04-14 07:19:34,121] Trial 1 finished with value: 14.949913341985459 and parameters: {'n_estimators': 124, 'max_depth': 40, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': None}. Best is trial 0 with value: 14.611247613427016.
[I 2024-04-14 07:20:04,476] Trial 2 finished with value: 15.66113735788673 and parameters: {'n_estimators': 242, 'max_depth': 50, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 0 with value: 14.611247613427016.
[I 2024-04-14 07:21:56,997] Trial 3 finished with value: 14.663847521663092 and parameters: {'n_estimators': 191, 'max_depth': 40, 'min_sa

[I 2024-04-14 07:48:51,020] Trial 33 finished with value: 14.293533100888574 and parameters: {'n_estimators': 161, 'max_depth': 15, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': None}. Best is trial 23 with value: 14.282572144609537.
[I 2024-04-14 07:50:13,328] Trial 34 finished with value: 14.471484817912222 and parameters: {'n_estimators': 198, 'max_depth': 20, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': None}. Best is trial 23 with value: 14.282572144609537.
[I 2024-04-14 07:51:05,102] Trial 35 finished with value: 14.541510623699184 and parameters: {'n_estimators': 127, 'max_depth': 35, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': None}. Best is trial 23 with value: 14.282572144609537.
[I 2024-04-14 07:51:55,932] Trial 36 finished with value: 14.963114265221991 and parameters: {'n_estimators': 245, 'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': None}. Best is trial 23 with value: 14.2825721446

[I 2024-04-14 08:17:49,212] Trial 66 finished with value: 19.844178948253596 and parameters: {'n_estimators': 221, 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_features': 'sqrt'}. Best is trial 23 with value: 14.282572144609537.
[I 2024-04-14 08:18:54,370] Trial 67 finished with value: 14.305264337299993 and parameters: {'n_estimators': 182, 'max_depth': 15, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': None}. Best is trial 23 with value: 14.282572144609537.
[I 2024-04-14 08:19:01,469] Trial 68 finished with value: 21.44601950428786 and parameters: {'n_estimators': 160, 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 23 with value: 14.282572144609537.
[I 2024-04-14 08:19:44,414] Trial 69 finished with value: 14.291095389801916 and parameters: {'n_estimators': 121, 'max_depth': 15, 'min_samples_split': 9, 'min_samples_leaf': 1, 'max_features': None}. Best is trial 23 with value: 14.28257214

[I 2024-04-14 08:38:20,390] Trial 99 finished with value: 14.962940860477863 and parameters: {'n_estimators': 126, 'max_depth': 10, 'min_samples_split': 9, 'min_samples_leaf': 1, 'max_features': None}. Best is trial 84 with value: 14.269266607076498.


In [14]:
best_params = study.best_params
print("Best parameters:", best_params)

# Train the best model
best_model = RandomForestRegressor(**best_params)
best_model.fit(X_train, y_train)



Best parameters: {'n_estimators': 126, 'max_depth': 15, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': None}


RandomForestRegressor(max_depth=15, max_features=None, min_samples_split=10,
                      n_estimators=126)

In [15]:

# Evaluate on test data
mse = mean_squared_error(y_test, best_model.predict(X_test))
r2 = best_model.score(X_test, y_test)

print("Test MSE:", mse)
print("Test R^2 score:", r2)


Test MSE: 13.860953859260146
Test R^2 score: 0.8437079846156729


In [ ]:
##clustering?

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score



In [18]:
X_train.head()

,Delivery_person_Age,Delivery_person_Ratings,Vehicle_condition,multiple_deliveries,Distance,Weatherconditions_Cloudy,Weatherconditions_Fog,Weatherconditions_Sandstorms,Weatherconditions_Stormy,Weatherconditions_Sunny,...,Type_of_vehicle_electric_scooter,Type_of_vehicle_motorcycle,Type_of_vehicle_scooter,Festival_NaN,Festival_No,Festival_Yes,City_Metropolitian,City_NaN,City_Semi-Urban,City_Urban
0,34,4.5,1,0.0,16.903,False,False,False,False,True,...,False,True,False,False,True,False,True,False,False,False
1,21,4.6,2,0.0,9.193,False,False,False,False,True,...,False,True,False,False,True,False,False,False,False,True
2,35,4.6,2,1.0,16.608,False,False,True,False,False,...,True,False,False,False,True,False,True,False,False,False
3,29,4.9,0,1.0,7.602,True,False,False,False,False,...,False,True,False,False,True,False,True,False,False,False
4,23,4.8,2,1.0,3.019,False,False,False,False,True,...,True,False,False,False,True,False,False,False,False,True


In [26]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

In [31]:
# Applying PCA for better visualization and potentially improved clustering
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Finding the optimal number of clusters using the Elbow method
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_pca)
    sse.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), sse, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Sum of squared distances')
plt.title('Elbow Method For Optimal k')
plt.show()

# Assume optimal clusters found to be 3
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_scaled)


AttributeError: 'NoneType' object has no attribute 'split'

In [28]:
pip install --upgrade numpy scipy scikit-learn pandas seaborn matplotlib


  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/3f/6b/5610004206cf7f8e7ad91c5a85a8c71b2f2f8051a0c0c4d5916b76d6cbb2/numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.0 kB 330.3 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/61.0 kB 435.7 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.0 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 295.3 kB/s eta 0:00:00
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/d4/a1/d4adf25b6d2bef8d0ad1682829dcfcba97f3f96bb5b6f137bc3e41003cc7/scipy-1.13.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.6 kB ? eta -:--:--
     -------------------------------------- 60.6/60.

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\USER\\anaconda3\\Lib\\site-packages\\~umpy.libs\\libopenblas64__v0.3.23-293-gc2f4bdbb-gcc_10_3_0-2bde3a66a51006b2b53eb373ff767a3f.dll'
Consider using the `--user` option or check the permissions.



In [30]:
X_train

,Delivery_person_Age,Delivery_person_Ratings,Vehicle_condition,multiple_deliveries,Distance,Weatherconditions_Cloudy,Weatherconditions_Fog,Weatherconditions_Sandstorms,Weatherconditions_Stormy,Weatherconditions_Sunny,...,Type_of_vehicle_electric_scooter,Type_of_vehicle_motorcycle,Type_of_vehicle_scooter,Festival_NaN,Festival_No,Festival_Yes,City_Metropolitian,City_NaN,City_Semi-Urban,City_Urban
0,34,4.5,1,0.0,16.903,False,False,False,False,True,...,False,True,False,False,True,False,True,False,False,False
1,21,4.6,2,0.0,9.193,False,False,False,False,True,...,False,True,False,False,True,False,False,False,False,True
2,35,4.6,2,1.0,16.608,False,False,True,False,False,...,True,False,False,False,True,False,True,False,False,False
3,29,4.9,0,1.0,7.602,True,False,False,False,False,...,False,True,False,False,True,False,True,False,False,False
4,23,4.8,2,1.0,3.019,False,False,False,False,True,...,True,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34797,39,4.7,1,1.0,4.718,False,False,False,True,False,...,False,True,False,False,True,False,False,False,False,True
34798,26,4.8,2,1.0,1.555,True,False,False,False,False,...,False,True,False,False,True,False,True,False,False,False
34799,37,4.7,0,1.0,9.056,True,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
34800,21,4.7,1,1.0,1.509,False,False,True,False,False,...,False,True,False,False,True,False,False,False,False,True


In [32]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [34]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1, 10],
    'epsilon': [0.01, 0.1, 1, 10],
    'degree': [2, 3, 4] 
}

svr = SVR()
grid_search = GridSearchCV(svr, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)



Fitting 3 folds for each of 864 candidates, totalling 2592 fits


In [ ]:
best_svr = grid_search.best_estimator_
y_pred = best_svr.predict(X_test_scaled)

# Calculating errors
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Printing results
print(f"The best model parameters: {grid_search.best_params_}")
print(f"Mean Squared Error on Test set: {mse}")
print(f"R-squared on Test set: {r2}")